In [1]:
import geopandas as gpd
import xarray as xr
from datetime import datetime as dt
from os import path
import numpy as np
from matplotlib.path import Path
from scipy.interpolate import interp1d
from pyxlma import coords

In [2]:
date_i_want = dt(2022, 6, 2)
tfm = xr.open_dataset(f'/Volumes/LtgSSD/tobac_saves/tobac_Save_{date_i_want.strftime("%Y%m%d")}/Track_features_merges_augmented2.zarr',
                        engine='zarr', chunks='auto')
polyline = gpd.read_file(f'/Volumes/LtgSSD/analysis/sam_polyline/{date_i_want.strftime("%Y-%m-%d")}.json')

In [3]:
tfm

<xarray.Dataset> Size: 2GB
Dimensions:                           (feature: 16210, time: 283, cell: 945,
                                       x: 1001, y: 1001, track: 770)
Coordinates:
  * cell                              (cell) int64 8kB -1 4 6 ... 5092 5107 5110
  * feature                           (feature) int64 130kB 1 2 ... 16209 16210
  * time                              (time) datetime64[ns] 2kB 2022-06-02T00...
  * track                             (track) float64 6kB 0.0 1.0 ... 769.0
  * x                                 (x) float64 8kB -2.5e+05 ... 2.5e+05
  * y                                 (y) float64 8kB -2.5e+05 ... 2.5e+05
Data variables: (12/48)
    avg_L2-MCMIPC                     (feature, time) float64 37MB dask.array<chunksize=(16210, 283), meta=np.ndarray>
    cell_child_feature_count          (cell) float64 8kB dask.array<chunksize=(945,), meta=np.ndarray>
    cell_parent_track_id              (cell) float64 8kB dask.array<chunksize=(945,), meta=np.ndarray>
    feature_area                      (feature) float64 130kB dask.array<chunksize=(16210,), meta=np.ndarray>
    feature_flash_count               (feature) float64 130kB dask.array<chunksize=(16210,), meta=np.ndarray>
    feature_flash_count_area_GT_4km   (feature) float64 130kB dask.array<chunksize=(16210,), meta=np.ndarray>
    ...                                ...
    max_reflectivity                  (feature) float64 130kB dask.array<chunksize=(16210,), meta=np.ndarray>
    min_L2-MCMIPC                     (feature, time) float64 37MB dask.array<chunksize=(16210, 283), meta=np.ndarray>
    segmentation_mask                 (time, y, x) float32 1GB dask.array<chunksize=(114, 402, 402), meta=np.ndarray>
    segmentation_mask_cell            (time, y, x) float32 1GB dask.array<chunksize=(114, 402, 402), meta=np.ndarray>
    std_L2-MCMIPC                     (feature, time) float64 37MB dask.array<chunksize=(16210, 283), meta=np.ndarray>
    track_child_cell_count            (track) float64 6kB dask.array<chunksize=(770,), meta=np.ndarray>
Attributes:
    center_lat:  29.47
    center_lon:  -95.08

In [4]:
this_feature = tfm.isel(feature=0)

0

In [3]:
polyline

,index,geometry
0,2022-06-02 13:27:29,"POLYGON ((-91.46126 29.35024, -91.9645 29.4573..."
1,2022-06-02 13:46:03,"POLYGON ((-91.44301 29.35029, -92.29954 29.520..."


In [4]:
lon_wide = np.arange(-98.3, -91+.005, .01)
lat_wide = np.arange(25.5, 30+.005, .01)
lon_wide, lat_wide = np.meshgrid(lon_wide, lat_wide)
all_seabreezes_wide = np.zeros((lon_wide.shape[0], lon_wide.shape[1], tfm.time.shape[0]))
seabreeze_indices = []
seabreeze_times = []
zero_indices = []
zero_times = []

In [5]:
radar_lat, radar_lon = 29.47, -95.08
tpcs = coords.TangentPlaneCartesianSystem(ctrLat=radar_lat, ctrLon=radar_lon, ctrAlt=0)
geosys = coords.GeographicSystem()
x2d, y2d = np.meshgrid(tfm.x.data, tfm.y.data)
grid_ecef_coords = tpcs.toECEF(x2d.flatten(), y2d.flatten(), np.zeros_like(x2d).flatten())
grid_lon, grid_lat, _ = geosys.fromECEF(*grid_ecef_coords)
grid_lon = grid_lon.reshape(x2d.shape)
grid_lat = grid_lat.reshape(x2d.shape)

tfm = tfm.assign({'lat' : (('x', 'y'), grid_lat), 'lon' : (('x', 'y'), grid_lon)})

all_seabreezes_ds = xr.zeros_like(tfm.segmentation_mask)

In [6]:
for i, time in enumerate(tfm.time.data):
    this_seabreeze = np.zeros_like(lon_wide)
    time_dt = np.array(time).astype('datetime64[s]').astype(dt).item()
    if time_dt in polyline['index'].values.astype(dt):
        seabreeze_indices.append(i)
        seabreeze_times.append(time)
        this_polyline = polyline[polyline['index'] == time_dt]['geometry'].values[0]
        this_polyline_mpl = Path(np.array(this_polyline.exterior.coords))
        this_seabreeze = this_polyline_mpl.contains_points(np.array([lon_wide.flatten(), lat_wide.flatten()]).T).reshape(lon_wide.shape)
        all_seabreezes_wide[:, :, i] = this_seabreeze
        this_seabreeze_ds = this_polyline_mpl.contains_points(np.array([grid_lon.flatten(), grid_lat.flatten()]).T).reshape(grid_lon.shape)
        all_seabreezes_ds[i, :, :] = this_seabreeze_ds
    else:
        zero_indices.append(i)
        zero_times.append(time)
seabreeze_indices = np.array(seabreeze_indices)
seabreeze_times = np.array(seabreeze_times)
zero_indices = np.array(zero_indices)
zero_times = np.array(zero_times)

In [7]:
all_seabreezes_ds

<xarray.DataArray 'segmentation_mask' (time: 283, y: 1001, x: 1001)> Size: 1GB
dask.array<setitem, shape=(283, 1001, 1001), dtype=float32, chunksize=(114, 402, 402), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2kB 2022-06-02T00:05:22.842000 ... 2022-06...
  * x        (x) float64 8kB -2.5e+05 -2.495e+05 -2.49e+05 ... 2.495e+05 2.5e+05
  * y        (y) float64 8kB -2.5e+05 -2.495e+05 -2.49e+05 ... 2.495e+05 2.5e+05

In [8]:
# Convert seabreeze_times and zero_times to numeric for interpolation
seabreeze_times_num = seabreeze_times.astype('float64')
zero_times_num = zero_times.astype('float64')

In [9]:
def interp_seabreeze_times(all_seabreezes, seabreeze_indices, seabreeze_times_num, zero_indices, zero_times_num):
    all_seabreezes = all_seabreezes.copy()
    # Interpolation: loop through each grid point (lat, lon) and interpolate over time
    for lat_idx in range(all_seabreezes.shape[0]):
        for lon_idx in range(all_seabreezes.shape[1]):
            # Get the seabreeze field values at valid times for this (lat, lon) grid point
            seabreeze_values = all_seabreezes[lat_idx, lon_idx, seabreeze_indices]
            
            # Skip interpolation if all values are zero
            if np.all(seabreeze_values == 0):
                continue
            
            # Create interpolator for the seabreeze field values based on actual times
            interpolator = interp1d(
                seabreeze_times_num,
                seabreeze_values,
                kind='linear',
                bounds_error=False,
                fill_value=0  # You can choose another fill method if appropriate
            )
            
            # Interpolate for missing times (all-zero slices)
            interpolated_values = interpolator(zero_times_num)
            
            # Update all_seabreezes with interpolated values at zero_indices
            all_seabreezes[lat_idx, lon_idx, zero_indices] = interpolated_values
    return all_seabreezes

In [10]:
interp_wide = interp_seabreeze_times(all_seabreezes_wide, seabreeze_indices, seabreeze_times_num, zero_indices, zero_times_num)

In [11]:
all_seabreezes_ds.shape

(283, 1001, 1001)

In [12]:
all_seabreezes_wide.shape

(451, 731, 283)

In [13]:
all_seabreezes_ds = interp_seabreeze_times(all_seabreezes_ds.T.compute(), seabreeze_indices, seabreeze_times_num, zero_indices, zero_times_num).T

In [32]:
tfm['seabreeze'] = all_seabreezes_ds

In [ ]:
feature_seabreeze = xr.zeros_like(tfm.feature)
cell_seabreeze = np.zeros_like(np.meshgrid(tfm.cell, tfm.time))
for i, feat_id in enumerate(tfm.feature.data):
    this_feat = tfm.sel(feature=feat_id)
    this_feat_lon = this_feat.feature_lon.compute()
    this_feat_lat = this_feat.feature_lat.compute()
    ll_dist = ((tfm.lon - this_feat_lon)**2 + (tfm.lat - this_feat_lat)**2)**(0.5)
    ll_dist_min = np.unravel_index(np.argmin(ll_dist.data.flatten()), ll_dist.shape)
    this_feat_time_index = this_feat.feature_time_index.data.compute().item()
    closest_seabreeze = tfm.seabreeze.isel(time=this_feat_time_index)[ll_dist_min]
    feature_seabreeze[i] = closest_seabreeze

In [73]:
tfm['feature_seabreeze'] = feature_seabreeze